In [1]:
# Parameters
alpha = 0.6
ratio = 0.1


In [2]:

import os
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Arc
from IPython.display import display
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import mezzala
import itertools
import json
import urllib.request
from datetime import datetime  
from collections import defaultdict
from collections import Counter
from scipy import stats
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from PIL import Image
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import unidecode
import re



In [3]:
# Definir la ruta relativa
data_path = "../../data_backup/archivos_parquet"

# Obtener la lista de archivos parquet que contienen "all" o "eventos"
parquet_files = [f for f in os.listdir(data_path) 
                 if f.endswith('.parquet') 
                 and ('all' in f.lower() or 'eventos' in f.lower())]

# Procesar cada archivo y crear variables globales
for file in parquet_files:
    # Crear el nombre del DataFrame (reemplazar .parquet y agregar prefijo df_)
    df_name = 'df_' + file.replace('.parquet', '')
    
    # Leer el archivo parquet
    file_path = os.path.join(data_path, file)
    
    # Crear la variable global usando globals()
    globals()[df_name] = pd.read_parquet(file_path)
    
    # Imprimir información sobre el proceso
    print(f"Archivo procesado: {file}")
    print(f"Nombre del DataFrame: {df_name}")
    print(f"Dimensiones: {globals()[df_name].shape}")
    print("-" * 50)

Archivo procesado: eventos_datos_acumulados.parquet
Nombre del DataFrame: df_eventos_datos_acumulados
Dimensiones: (13974, 191)
--------------------------------------------------
Archivo procesado: seasons_teams_all.parquet
Nombre del DataFrame: df_seasons_teams_all
Dimensiones: (105, 7)
--------------------------------------------------
Archivo procesado: formations_league_all.parquet
Nombre del DataFrame: df_formations_league_all
Dimensiones: (44035, 9)
--------------------------------------------------


Archivo procesado: eventos_metricas_alaves.parquet
Nombre del DataFrame: df_eventos_metricas_alaves
Dimensiones: (810068, 134)
--------------------------------------------------
Archivo procesado: names_players_league_all.parquet
Nombre del DataFrame: df_names_players_league_all
Dimensiones: (9619, 12)
--------------------------------------------------
Archivo procesado: df_eventos_acumulados_sin_KPI.parquet
Nombre del DataFrame: df_df_eventos_acumulados_sin_KPI
Dimensiones: (14650, 180)
--------------------------------------------------
Archivo procesado: leagues_all.parquet
Nombre del DataFrame: df_leagues_all
Dimensiones: (42, 7)
--------------------------------------------------
Archivo procesado: sequence_data_league_all.parquet
Nombre del DataFrame: df_sequence_data_league_all
Dimensiones: (263349, 7)
--------------------------------------------------
Archivo procesado: players_league_all.parquet
Nombre del DataFrame: df_players_league_all
Dimensiones: (9642, 6)
-----------------

Archivo procesado: matches_league_all.parquet
Nombre del DataFrame: df_matches_league_all
Dimensiones: (1001, 33)
--------------------------------------------------
Archivo procesado: lineups_league_all.parquet
Nombre del DataFrame: df_lineups_league_all
Dimensiones: (16793, 12)
--------------------------------------------------
Archivo procesado: player_stats_extended_league_all.parquet
Nombre del DataFrame: df_player_stats_extended_league_all
Dimensiones: (14532, 79)
--------------------------------------------------
Archivo procesado: teams_league_all.parquet
Nombre del DataFrame: df_teams_league_all
Dimensiones: (263, 6)
--------------------------------------------------


In [4]:
# Paso 1: Realizar el merge inicial
df_combined = pd.merge(
    df_player_stats_extended_league_all,
    df_df_eventos_acumulados_sin_KPI[['player_id','team_id', 'equipo', 'jugador', 'demarcacion', 'dorsal', 'liga', 'jornada', 'partido']],
    on='player_id',
    how='left'
)

# Paso 2: Merge con df_lineups_league_all para obtener is_starting_lineup
df_combined = pd.merge(
    df_combined,
    df_lineups_league_all[['match_id', 'season_id', 'player_id', 'is_starting_lineup']],
    on=['match_id', 'season_id', 'player_id'],
    how='left'
)

# Paso 3: Crear la columna starter_sub
df_combined['starter_sub'] = np.where(
    df_combined['is_starting_lineup'] == True,
    'starter',
    'sub'
)

# Paso 4: Reemplazar valores nulos por 0 (excepto en starter_sub)
for column in df_combined.columns:
    if column != 'starter_sub':
        df_combined[column] = df_combined[column].fillna(0)

# Paso 5: Eliminar filas duplicadas
df_combined = df_combined.drop_duplicates()

# Verificar las columnas del DataFrame final
df_combined

,goal,total_shot,shot_on_target,shot_off_target,shot_blocked,shot_in_PA,shot_outside_of_PA,free_kick,corner_kick,throw_in,...,team_id_y,equipo,jugador,demarcacion,dorsal,liga,jornada,partido,is_starting_lineup,starter_sub
0,0,3,1,2,0,3,0,4,0,0,...,7897.0,Deportivo Alavés Cad. A,Unax Ariza Fernandez,Mediocentro,10,Liga Vasca Cadete,Jornada 3,Romo Cad. A 2 - 1 Deportivo Alavés Cad. A,True,starter
1,0,3,1,2,0,3,0,4,0,0,...,7897.0,Deportivo Alavés Cad. A,Unax Ariza Fernandez,Mediocentro,10,Copa Vasca Cadete,Cuartos de final,Deportivo Alavés Cad. A 2 - 1 SD Eibar Cad. A,True,starter
2,1,2,1,1,0,2,0,0,0,0,...,7897.0,Deportivo Alavés Cad. A,Biniam Gonzalez,Extremo Izquierdo,11,Liga Vasca Cadete,Jornada 3,Romo Cad. A 2 - 1 Deportivo Alavés Cad. A,True,starter
3,1,2,1,1,0,2,0,0,0,0,...,7897.0,Deportivo Alavés Cad. A,Biniam Gonzalez,Extremo Izquierdo,11,Copa Vasca Cadete,Cuartos de final,Deportivo Alavés Cad. A 2 - 1 SD Eibar Cad. A,True,starter
4,0,0,0,0,0,0,0,1,0,0,...,7897.0,Deportivo Alavés Cad. A,Aitor Suberviola,Mediocentro,17,Copa Vasca Cadete,Cuartos de final,Deportivo Alavés Cad. A 2 - 1 SD Eibar Cad. A,False,sub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215517,0,3,1,2,0,1,2,1,4,0,...,13051.0,Aretxabaleta KE,Josu Aiizpurua Olabe,Mediocentro,21,Tercera RFEF - G4,Jornada 27,Aretxabaleta KE 0 - 2 Deportivo Alavés C,True,starter
215518,0,0,0,0,0,0,0,1,0,0,...,13051.0,Aretxabaleta KE,Roldan Guirao Mikel,Mediocentro Defensivo,30,Tercera RFEF - G4,Jornada 27,Aretxabaleta KE 0 - 2 Deportivo Alavés C,False,sub
215519,0,0,0,0,0,0,0,0,0,0,...,13051.0,Aretxabaleta KE,Lasa Sanz Jagoba,Mediocentro,16,Tercera RFEF - G4,Jornada 27,Aretxabaleta KE 0 - 2 Deportivo Alavés C,False,sub
215520,0,2,0,1,1,1,1,0,0,0,...,13051.0,Aretxabaleta KE,Sanchez Iparragirre Unax,Mediocentro Defensivo,14,Tercera RFEF - G4,Jornada 27,Aretxabaleta KE 0 - 2 Deportivo Alavés C,True,starter


In [5]:

# Lista de columnas a mantener
columnas_a_mantener = [
    'goal', 'match_id','total_shot', 'shot_on_target', 'shot_in_PA', 'shot_outside_of_PA',
    'free_kick', 'penalty_kick', 'corner_kick', 'pass', 'pass_succeeded',
    'assist', 'key_pass', 'control_under_pressure', 'cross',
    'cross_succeeded', 'final_third_area_pass', 'final_third_area_pass_succeeded',
    'middle_area_pass', 'middle_area_pass_succeeded', 'defensive_area_pass',
    'defensive_area_pass_succeeded', 'long_pass', 'long_pass_succeeded',
    'forward_pass', 'forward_pass_succeeded', 'backward_pass',
    'backward_pass_succeeded', 'sideways_pass', 'sideways_pass_succeeded',
    'tackle', 'tackle_succeeded', 'aerial_duel', 'aerial_duel_succeeded',
    'ground_duel', 'ground_duel_succeeded', 'loose_ball_duel',
    'loose_ball_duel_succeeded', 'block', 'intercept', 'clearance', 'recovery',
    'intervention', 'take_on', 'take_on_succeeded', 'mistake',
    'foul', 'offside', 'yellow_card', 'red_card', 'own_goal',
    'goal_conceded', 'aerial_clearance', 'aerial_clearance_succeeded',
    'defensive_line_support', 'defensive_line_support_succeeded',
    'rating', 'save_by_catching', 'save_by_punching','play_time'
]



# Añadir las columnas que agregamos en los pasos anteriores
columnas_adicionales = ['player_id', 'partido', 'jornada', 'season_id', 'jugador','demarcacion', 'liga', 'dorsal', 'equipo', 'starter_sub']

# Combinar todas las columnas que queremos mantener
todas_columnas = columnas_a_mantener + columnas_adicionales

# Seleccionar solo estas columnas de df_combined y filtrar por defensas centrales
df_players = df_combined[todas_columnas]

# Convertir play_time de milésimas de segundos a minutos enteros
df_players['play_time_minutes'] = np.round(df_players['play_time'] / (1000 * 60)).astype(int)

# Contar el número de valores nulos por columna antes de reemplazarlos
null_counts_before = df_combined.isnull().sum()
print("Número de valores nulos por columna antes de reemplazarlos:")
print(null_counts_before)
print("\n")

# Reemplazar valores nulos por 0 (excepto en back_number)
for column in df_combined.columns:
    if column != 'back_number':
        df_combined[column] = df_combined[column].fillna(0)
        

# Mostrar información sobre el DataFrame
df_players

Número de valores nulos por columna antes de reemplazarlos:
goal                  0
total_shot            0
shot_on_target        0
shot_off_target       0
shot_blocked          0
                     ..
liga                  0
jornada               0
partido               0
is_starting_lineup    0
starter_sub           0
Length: 89, dtype: int64




/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_3846/2122564498.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_players['play_time_minutes'] = np.round(df_players['play_time'] / (1000 * 60)).astype(int)


,goal,match_id,total_shot,shot_on_target,shot_in_PA,shot_outside_of_PA,free_kick,penalty_kick,corner_kick,pass,...,partido,jornada,season_id,jugador,demarcacion,liga,dorsal,equipo,starter_sub,play_time_minutes
0,0,93857,3,1,3,0,4,0,0,30,...,Romo Cad. A 2 - 1 Deportivo Alavés Cad. A,Jornada 3,3001,Unax Ariza Fernandez,Mediocentro,Liga Vasca Cadete,10,Deportivo Alavés Cad. A,starter,80
1,0,93857,3,1,3,0,4,0,0,30,...,Deportivo Alavés Cad. A 2 - 1 SD Eibar Cad. A,Cuartos de final,3001,Unax Ariza Fernandez,Mediocentro,Copa Vasca Cadete,10,Deportivo Alavés Cad. A,starter,80
2,1,93857,2,1,2,0,0,0,0,21,...,Romo Cad. A 2 - 1 Deportivo Alavés Cad. A,Jornada 3,3001,Biniam Gonzalez,Extremo Izquierdo,Liga Vasca Cadete,11,Deportivo Alavés Cad. A,starter,80
3,1,93857,2,1,2,0,0,0,0,21,...,Deportivo Alavés Cad. A 2 - 1 SD Eibar Cad. A,Cuartos de final,3001,Biniam Gonzalez,Extremo Izquierdo,Copa Vasca Cadete,11,Deportivo Alavés Cad. A,starter,80
4,0,93857,0,0,0,0,1,0,0,3,...,Deportivo Alavés Cad. A 2 - 1 SD Eibar Cad. A,Cuartos de final,3001,Aitor Suberviola,Mediocentro,Copa Vasca Cadete,17,Deportivo Alavés Cad. A,sub,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215517,0,172798,3,1,1,2,1,0,4,20,...,Aretxabaleta KE 0 - 2 Deportivo Alavés C,Jornada 27,4104,Josu Aiizpurua Olabe,Mediocentro,Tercera RFEF - G4,21,Aretxabaleta KE,starter,69
215518,0,172798,0,0,0,0,1,0,0,7,...,Aretxabaleta KE 0 - 2 Deportivo Alavés C,Jornada 27,4104,Roldan Guirao Mikel,Mediocentro Defensivo,Tercera RFEF - G4,30,Aretxabaleta KE,sub,10
215519,0,172798,0,0,0,0,0,0,0,9,...,Aretxabaleta KE 0 - 2 Deportivo Alavés C,Jornada 27,4104,Lasa Sanz Jagoba,Mediocentro,Tercera RFEF - G4,16,Aretxabaleta KE,sub,21
215520,0,172798,2,0,1,1,0,0,0,35,...,Aretxabaleta KE 0 - 2 Deportivo Alavés C,Jornada 27,4104,Sanchez Iparragirre Unax,Mediocentro Defensivo,Tercera RFEF - G4,14,Aretxabaleta KE,starter,80


In [6]:
def create_kpi_players(df_players):
    print("Iniciando la creación de df_KPI_players...")
    
    # Hacer una copia completa del DataFrame original
    df_KPI_players = df_players.copy()
    
    df_KPI_players['participacion'] = df_KPI_players.apply(lambda row:
        (10 if row['play_time_minutes'] >= 90 else
         0 if row['play_time_minutes'] == 0 else
         (row['play_time_minutes'] / 90) * 10) +
        (3 if row['starter_sub'] == 'starter' else 0),
        axis=1
    ).round(2)
    
    def safe_divide(a, b):
        return np.round(np.where(b != 0, a / b, 0), 2)
    
    # Métricas de tiro
    df_KPI_players['%_tiros_dentro_area'] = safe_divide(df_players['shot_in_PA'], df_players['total_shot'])
    df_KPI_players['%_tiros_fuera_area'] = safe_divide(df_players['shot_outside_of_PA'], df_players['total_shot'])
    df_KPI_players['Capacidad_disparos'] = df_KPI_players['%_tiros_fuera_area'] + df_KPI_players['%_tiros_dentro_area']
    
    # Métricas de pase y creatividad
    df_KPI_players['lanzador_ABP'] = (df_players['free_kick'] + df_players['corner_kick'] + df_players['penalty_kick']).round(2)
    df_KPI_players['precision_pase'] = safe_divide(df_players['pass_succeeded'], df_players['pass'])
    df_KPI_players['influencia_pases_gol'] = (df_players['assist'] + df_players['key_pass'] + df_players['cross_succeeded']).round(2)
    df_KPI_players['controles_bajo_presion'] = df_players['control_under_pressure'].round(2)
    
    # Métricas de eficacia
    df_KPI_players['Eficacia_disparos'] = safe_divide(df_players['shot_on_target'], df_players['total_shot'])
    df_KPI_players['Eficacia_goleadora'] = safe_divide(df_players['goal'], df_players['shot_on_target'])
    df_KPI_players['Habilidad_regate'] = safe_divide(df_players['take_on_succeeded'], df_players['take_on'])
    
    # Métricas de construcción de juego
    df_KPI_players['influencia_finalización'] = safe_divide(df_players['final_third_area_pass_succeeded'], df_players['final_third_area_pass'])
    df_KPI_players['influencia_construccion'] = safe_divide(
        (df_players['middle_area_pass_succeeded'] + df_players['defensive_area_pass_succeeded']),
        (df_players['middle_area_pass'] + df_players['defensive_area_pass'])
    )
    df_KPI_players['precision_pases_largos'] = safe_divide(df_players['long_pass_succeeded'], df_players['long_pass'])
    df_KPI_players['verticalidad_pase'] = (
        ((df_players['forward_pass'] - (df_players['backward_pass'] + df_players['sideways_pass'])) * df_players['forward_pass_succeeded'])
    )
    
    # Métricas defensivas
    df_KPI_players['efectividad_entradas'] = safe_divide(df_players['tackle_succeeded'], df_players['tackle'])
    df_KPI_players['efectividad_juego_aereo'] = safe_divide(df_players['aerial_duel_succeeded'], df_players['aerial_duel'])
    df_KPI_players['efectividad_duelos_suelo'] = safe_divide(df_players['ground_duel_succeeded'], df_players['ground_duel'])
    df_KPI_players['efectividad_balones_divididos'] = safe_divide(df_players['loose_ball_duel_succeeded'], df_players['loose_ball_duel'])
    
    df_KPI_players['contundencia'] = (df_KPI_players['efectividad_balones_divididos'] + 
                                     df_KPI_players['efectividad_duelos_suelo'] + 
                                     df_KPI_players['efectividad_juego_aereo'] + 
                                     df_KPI_players['efectividad_entradas']).round(2)
    
    df_KPI_players['capacidad_defensiva'] = safe_divide(
        (df_players['intercept'] + df_players['clearance'] + df_players['tackle'] + 
         df_players['block'] + df_players['ground_duel'] + df_players['loose_ball_duel'] +
         df_players['recovery'] + df_players['save_by_catching'] + df_players['save_by_punching']), 
        df_players['intervention']
    )
    
    df_KPI_players['capacidad_juego_aereo_def'] = safe_divide(
        safe_divide(df_players['aerial_clearance_succeeded'], df_players['aerial_clearance']), 
        df_players['clearance']
    )
    
    df_KPI_players['rendimiento_posicional_def'] = safe_divide(
        df_players['defensive_line_support_succeeded'], 
        df_players['defensive_line_support']
    )
    
    # Métricas de valoración y errores
    df_KPI_players['calificacion_Bepro'] = df_players['rating'].round(2)
    df_KPI_players['errores'] = df_players['mistake'].round(2)
    df_KPI_players['faltas'] = df_players['foul'].round(2)
    df_KPI_players['fuera_juego'] = df_players['offside'].round(2)
    df_KPI_players['tarj_amarilla'] = df_players['yellow_card'].round(2)
    df_KPI_players['tarj_roja'] = df_players['red_card'].round(2)
    df_KPI_players['gol_en_propia'] = df_players['own_goal'].round(2)
    df_KPI_players['gol_en_contra'] = df_players['goal_conceded'].round(2)
    
    # Talento (métrica compuesta)
    df_KPI_players['talento'] = (
        ((df_KPI_players['Habilidad_regate'] + safe_divide(df_players['shot_on_target'], df_players['total_shot'])) + 
         df_players['control_under_pressure'] + df_players['free_kick'] + df_players['corner_kick'] + 
         df_players['penalty_kick'] + df_players['pass_succeeded'] + df_players['goal'] + 
         df_players['assist'] + df_players['key_pass'] + df_players['cross_succeeded'] +
         df_players['take_on_succeeded'] + df_players['tackle_succeeded'] + 
         df_players['aerial_duel_succeeded'] + df_players['ground_duel_succeeded'] + 
         df_players['loose_ball_duel_succeeded'] + df_players['rating'] + 
         df_players['forward_pass_succeeded'] + df_players['save_by_catching'] + 
         df_players['save_by_punching']) - 
        (df_players['mistake'] + df_players['foul'] + df_players['own_goal'] + 
         df_players['goal_conceded'] + df_players['offside'])
    )
    
    # Redondear todas las columnas numéricas a 2 decimales
    df_KPI_players = df_KPI_players.round(2)
    
    print("df_KPI_players creado exitosamente.")
    return df_KPI_players

try:
    df_KPI_players = create_kpi_players(df_players)
    print("\nDimensiones del DataFrame:", df_KPI_players.shape)
except Exception as e:
    print(f"Error al crear df_KPI_players: {e}")

Iniciando la creación de df_KPI_players...


df_KPI_players creado exitosamente.

Dimensiones del DataFrame: (215461, 103)


In [7]:
# 1. Creamos una copia del DataFrame original
KPI_jugador_liga = df_KPI_players.copy()

# 2. Aplicamos el mapeo de ligas
def map_league(liga):
    if liga in ['Copa Vasca Cadete', 'Liga Vasca Cadete']:
        return 'Liga Vasca Cadete'
    elif liga in ['División de Honor - Araba', 'Tercera RFEF - G4']:
        return 'Tercera RFEF'
    else:
        return liga

In [8]:
df_KPI_players

,goal,match_id,total_shot,shot_on_target,shot_in_PA,shot_outside_of_PA,free_kick,penalty_kick,corner_kick,pass,...,rendimiento_posicional_def,calificacion_Bepro,errores,faltas,fuera_juego,tarj_amarilla,tarj_roja,gol_en_propia,gol_en_contra,talento
0,0,93857,3,1,3,0,4,0,0,30,...,0.0,8.14,2,2,1,0,0,0,0,57.47
1,0,93857,3,1,3,0,4,0,0,30,...,0.0,8.14,2,2,1,0,0,0,0,57.47
2,1,93857,2,1,2,0,0,0,0,21,...,0.0,7.66,1,2,1,0,0,0,0,22.16
3,1,93857,2,1,2,0,0,0,0,21,...,0.0,7.66,1,2,1,0,0,0,0,22.16
4,0,93857,0,0,0,0,1,0,0,3,...,0.0,6.71,0,1,0,0,0,0,0,12.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215517,0,172798,3,1,1,2,1,0,4,20,...,0.0,6.98,0,3,1,0,0,0,0,35.31
215518,0,172798,0,0,0,0,1,0,0,7,...,0.0,6.36,1,0,0,0,0,0,0,19.36
215519,0,172798,0,0,0,0,0,0,0,9,...,0.0,5.85,0,0,0,0,0,0,0,12.85
215520,0,172798,2,0,1,1,0,0,0,35,...,0.0,6.51,0,2,0,1,0,0,0,48.51


In [9]:
def normalize_by_league(data, df_liga, player_ids, kpi_name=None, min_val=0, max_val=10):
    if not isinstance(data, pd.Series):
        data = pd.Series(data, index=df_liga.index)
    
    temp_df = pd.DataFrame({
        'value': data,
        'liga': df_liga,
        'player_id': player_ids
    })
    
    normalized = np.zeros_like(data, dtype=float)
    
    # Definir configuración específica para cada KPI
    kpi_configs = {
        # KPIs con muchos ceros: aplicar penalización suave y multiplicación
        'KPI_peligro_generado': {'multiplier': 3, 'base_min': 4, 'base_max': 8, 'zero_penalty': 0.5, 'zero_replacement': 0.5},
        'KPI_finalizacion': {'multiplier': 3, 'base_min': 4, 'base_max': 8, 'zero_penalty': 0.5, 'zero_replacement': 0.5},
        'KPI_juego_aereo': {'multiplier': 3, 'base_min': 4, 'base_max': 8, 'zero_penalty': 0.5, 'zero_replacement': 0.5},
        'KPI_capacidad_recuperacion': {'multiplier': 4, 'base_min': 4, 'base_max': 8, 'zero_penalty': 0.5, 'zero_replacement': 0.5},
        'KPI_eficacia_defensiva': {'multiplier': 4, 'base_min': 4, 'base_max': 8, 'zero_penalty': 0.5, 'zero_replacement': 0.5},
        
        # KPIs sin muchos ceros: no aplicar multiplicación
        'KPI_construccion_ataque': {'multiplier': 1, 'base_min': 5, 'base_max': 9, 'zero_penalty': 0.2, 'zero_replacement': 0},
        'KPI_progresion': {'multiplier': 1, 'base_min': 5, 'base_max': 9, 'zero_penalty': 0.2, 'zero_replacement': 0},
        'KPI_habilidad_individual': {'multiplier': 1, 'base_min': 5, 'base_max': 9, 'zero_penalty': 0.2, 'zero_replacement': 0},
        'KPI_condicional': {'multiplier': 1, 'base_min': 6, 'base_max': 10, 'zero_penalty': 0.1, 'zero_replacement': 0},
        'KPI_posicionamiento_tactico': {'multiplier': 1, 'base_min': 6, 'base_max': 10, 'zero_penalty': 0.1, 'zero_replacement': 0}
    }
    
    # Obtener configuración para el KPI actual
    config = kpi_configs.get(kpi_name, {'multiplier': 1, 'base_min': 5, 'base_max': 9, 'zero_penalty': 0.2, 'zero_replacement': 0})
    
    for league in temp_df['liga'].unique():
        mask = (temp_df['liga'] == league)
        league_data = temp_df.loc[mask].copy()
        
        if len(league_data) > 0:
            # Reemplazar ceros con un valor ligeramente superior
            zero_mask = league_data['value'] == 0
            if zero_mask.any():
                league_data.loc[zero_mask, 'value'] = config['zero_replacement']
            
            # Aplicar multiplicador a los valores no cero si es necesario
            non_zero_mask = league_data['value'] != 0
            if non_zero_mask.any() and config['multiplier'] > 1:
                league_data.loc[non_zero_mask, 'value'] *= config['multiplier']
            
            # Calcular proporción de ceros por jugador
            player_stats = league_data.groupby('player_id')['value'].agg([
                ('zero_prop', lambda x: (x == 0).mean()),
                ('non_zero_mean', lambda x: x[x != 0].mean() if (x != 0).any() else 0)
            ])
            
            # Calcular factor de penalización por ceros
            player_stats['penalty_factor'] = 1 - (player_stats['zero_prop'] * config['zero_penalty'])
            
            # Procesar cada jugador
            for player_id in league_data['player_id'].unique():
                player_mask = (league_data['player_id'] == player_id)
                player_data = league_data.loc[player_mask, 'value']
                
                if len(player_data) > 0:
                    non_zero_mask = player_data != 0
                    if non_zero_mask.any():
                        non_zero_values = player_data[non_zero_mask]
                        penalty = player_stats.loc[player_id, 'penalty_factor']
                        
                        # Normalizar valores no cero
                        ranks = stats.rankdata(non_zero_values)
                        min_rank, max_rank = ranks.min(), ranks.max()
                        
                        if max_rank > min_rank:
                            normalized_ranks = (ranks - min_rank) / (max_rank - min_rank)
                            base_range = config['base_max'] - config['base_min']
                            adjusted_values = config['base_min'] + (normalized_ranks * base_range)
                            
                            # Aplicar penalización por ceros
                            adjusted_values = adjusted_values * penalty
                            
                            league_data.loc[player_mask & non_zero_mask, 'value'] = adjusted_values
                
            normalized[mask] = league_data['value'].values
    
    normalized = np.clip(normalized, min_val, max_val)
    return pd.Series(normalized, index=data.index)

def calculate_kpis_normalized(df):
    df = df.copy()
    
    def map_league(liga):
        if liga in ['Copa Vasca Cadete', 'Liga Vasca Cadete']:
            return 'Liga Vasca Cadete'
        elif liga in ['División de Honor - Araba', 'Tercera RFEF - G4']:
            return 'Tercera RFEF'
        else:
            return liga
    
    df['liga'] = df['liga'].apply(map_league)
    existing_columns = df.columns.tolist()
    kpi_columns = []
    
    def calculate_kpi_per_minute(values, play_time):
        return np.where(play_time > 0, values / play_time, 0)

    # KPI 1: Condicional
    if all(col in existing_columns for col in ['participacion', 'capacidad_defensiva', 
                                             'capacidad_juego_aereo_def', 'play_time_minutes']):
        condicional = calculate_kpi_per_minute(
            (df['participacion'] * 4 +
             df['capacidad_defensiva'] * 3 + 
             df['capacidad_juego_aereo_def'] * 3),
            df['play_time_minutes']
        )
        df['KPI_condicional'] = normalize_by_league(condicional, df['liga'], df['player_id'], 'KPI_condicional')
        kpi_columns.append('KPI_condicional')

    # KPI 2: Construcción de Ataque
    if all(col in existing_columns for col in ['influencia_construccion', 'precision_pase', 
                                             'precision_pases_largos', 'verticalidad_pase', 
                                             'participacion', 'play_time_minutes']):
        construccion_ataque = calculate_kpi_per_minute(
            ((df['influencia_construccion'] * 4 +
              df['precision_pase'] * 2 +
              df['precision_pases_largos'] * 2 +
              df['verticalidad_pase'] * 2) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_construccion_ataque'] = normalize_by_league(construccion_ataque, df['liga'], df['player_id'], 'KPI_construccion_ataque')
        kpi_columns.append('KPI_construccion_ataque')

    # KPI 3: Progresión
    if all(col in existing_columns for col in ['verticalidad_pase', 'precision_pase', 
                                             'precision_pases_largos', 'participacion', 
                                             'play_time_minutes']):
        progresion = calculate_kpi_per_minute(
            ((df['verticalidad_pase'] * 6 +
              df['precision_pase'] * 2 +
              df['precision_pases_largos'] * 2) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_progresion'] = normalize_by_league(progresion, df['liga'], df['player_id'], 'KPI_progresion')
        kpi_columns.append('KPI_progresion')

    # KPI 4: Habilidad Individual
    if all(col in existing_columns for col in ['talento', 'controles_bajo_presion', 
                                             'participacion', 'play_time_minutes']):
        habilidad_individual = calculate_kpi_per_minute(
            ((df['talento'] * 8 +
              df['controles_bajo_presion'] * 2) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_habilidad_individual'] = normalize_by_league(habilidad_individual, df['liga'], df['player_id'], 'KPI_habilidad_individual')
        kpi_columns.append('KPI_habilidad_individual')

    # KPI 5: Peligro Generado
    if all(col in existing_columns for col in ['influencia_pases_gol', 'participacion', 
                                             'Capacidad_disparos', 'play_time_minutes']):
        peligro_generado = calculate_kpi_per_minute(
            ((df['influencia_pases_gol'] * 6 +
              df['Capacidad_disparos'] * 4) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_peligro_generado'] = normalize_by_league(peligro_generado, df['liga'], df['player_id'], 'KPI_peligro_generado')
        kpi_columns.append('KPI_peligro_generado')

    # KPI 6: Finalización
    if all(col in existing_columns for col in ['Eficacia_disparos', 'Eficacia_goleadora', 
                                             'influencia_finalización', 'participacion', 
                                             'play_time_minutes']):
        finalizacion = calculate_kpi_per_minute(
            (((df['Eficacia_disparos'] + df['Eficacia_goleadora']) * 6 +
              df['influencia_finalización'] * 4) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_finalizacion'] = normalize_by_league(finalizacion, df['liga'], df['player_id'], 'KPI_finalizacion')
        kpi_columns.append('KPI_finalizacion')

    # KPI 7: Eficacia Defensiva
    if all(col in existing_columns for col in ['efectividad_entradas', 'capacidad_defensiva', 
                                             'rendimiento_posicional_def', 'participacion', 
                                             'faltas', 'play_time_minutes']):
        eficacia_defensiva = calculate_kpi_per_minute(
            (((df['efectividad_entradas'] - df['faltas']) * 3 +
              df['capacidad_defensiva'] * 4 +
              df['rendimiento_posicional_def'] * 3) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_eficacia_defensiva'] = normalize_by_league(eficacia_defensiva, df['liga'], df['player_id'], 'KPI_eficacia_defensiva')
        kpi_columns.append('KPI_eficacia_defensiva')

    # KPI 8: Juego Aéreo
    if all(col in existing_columns for col in ['efectividad_juego_aereo', 'capacidad_juego_aereo_def', 
                                             'participacion', 'play_time_minutes']):
        juego_aereo = calculate_kpi_per_minute(
            ((df['efectividad_juego_aereo'] * 7 +
              df['capacidad_juego_aereo_def'] * 3) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_juego_aereo'] = normalize_by_league(juego_aereo, df['liga'], df['player_id'], 'KPI_juego_aereo')
        kpi_columns.append('KPI_juego_aereo')

    # KPI 9: Capacidad de Recuperación
    if all(col in existing_columns for col in ['capacidad_defensiva', 'efectividad_balones_divididos', 
                                             'participacion', 'play_time_minutes']):
        capacidad_recuperacion = calculate_kpi_per_minute(
            ((df['capacidad_defensiva'] * 7 +
              df['efectividad_balones_divididos'] * 3) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_capacidad_recuperacion'] = normalize_by_league(capacidad_recuperacion, df['liga'], df['player_id'], 'KPI_capacidad_recuperacion')
        kpi_columns.append('KPI_capacidad_recuperacion')

    # KPI 10: Posicionamiento Táctico
    if all(col in existing_columns for col in ['rendimiento_posicional_def', 'fuera_juego', 
                                             'capacidad_defensiva', 'participacion', 
                                             'play_time_minutes']):
        posicionamiento_tactico = calculate_kpi_per_minute(
            ((df['rendimiento_posicional_def'] * 5 +
              (1 - df['fuera_juego'] / df['participacion'].replace(0, np.nan)) * 3 +
              df['capacidad_defensiva'] * 2) * df['participacion']),
            df['play_time_minutes']
        )
        df['KPI_posicionamiento_tactico'] = normalize_by_league(posicionamiento_tactico, df['liga'], df['player_id'], 'KPI_posicionamiento_tactico')
        kpi_columns.append('KPI_posicionamiento_tactico')

    # KPI Rendimiento
    if kpi_columns and 'calificacion_Bepro' in existing_columns:
        df['calificacion_Bepro_norm'] = normalize_by_league(
            df['calificacion_Bepro'], 
            df['liga'],
            df['player_id']
        )
        
        all_metrics = kpi_columns + ['calificacion_Bepro_norm']
        
        # Pesos para cada tipo de KPI
        weights = {
            'KPI_construccion_ataque': 1.0,
            'KPI_progresion': 1.0,
            'KPI_condicional': 1.1,
            'KPI_habilidad_individual': 1.0,
            'KPI_peligro_generado': 1.0,
            'KPI_finalizacion': 1.0,
            'KPI_eficacia_defensiva': 1.2,
            'KPI_juego_aereo': 1.0,
            'KPI_capacidad_recuperacion': 1.2,
            'KPI_posicionamiento_tactico': 1.1,
            'calificacion_Bepro_norm': 1.0
        }
        
        # Calcular media ponderada
        weighted_sum = 0
        total_weight = 0
        for metric in all_metrics:
            weight = weights.get(metric, 1.0)
            weighted_sum += df[metric] * weight
            total_weight += weight
        
        df['KPI_rendimiento'] = weighted_sum / total_weight
        
        # Ajustes finales más suaves
        bonus_count = df[all_metrics].gt(7).sum(axis=1)
        penalty_count = df[all_metrics].lt(3).sum(axis=1)
        
        df['KPI_rendimiento'] += bonus_count * 0.1  # Ajuste más suave
        df['KPI_rendimiento'] -= penalty_count * 0.05  # Ajuste más suave
        
        df['KPI_rendimiento'] = np.clip(df['KPI_rendimiento'], 0, 10)
        kpi_columns.append('KPI_rendimiento')
    
    base_columns = ['jugador', 'equipo', 'player_id', 'match_id', 'demarcacion', 'liga']
    existing_base_columns = [col for col in base_columns if col in df.columns]
    selected_columns = existing_base_columns + kpi_columns
    
    return df[selected_columns].copy()

# Uso
df_eventos_con_KPI = calculate_kpis_normalized(KPI_jugador_liga)

In [10]:
df_eventos_con_KPI

,jugador,equipo,player_id,match_id,demarcacion,liga,KPI_condicional,KPI_construccion_ataque,KPI_progresion,KPI_habilidad_individual,KPI_peligro_generado,KPI_finalizacion,KPI_eficacia_defensiva,KPI_juego_aereo,KPI_capacidad_recuperacion,KPI_posicionamiento_tactico,KPI_rendimiento
0,Unax Ariza Fernandez,Deportivo Alavés Cad. A,121382,93857,Mediocentro,Liga Vasca Cadete,6.000000,9.00000,9.0,9.0,8.0000,8.000000,4.0000,4.000000,4.00000,6.000000,7.393103
1,Unax Ariza Fernandez,Deportivo Alavés Cad. A,121382,93857,Mediocentro,Liga Vasca Cadete,6.000000,9.00000,9.0,9.0,8.0000,8.000000,4.0000,4.000000,4.00000,6.000000,7.393103
2,Biniam Gonzalez,Deportivo Alavés Cad. A,251457,93857,Extremo Izquierdo,Liga Vasca Cadete,10.000000,5.00000,5.0,5.0,4.0000,8.000000,8.0000,4.000000,8.00000,10.000000,7.600000
3,Biniam Gonzalez,Deportivo Alavés Cad. A,251457,93857,Extremo Izquierdo,Liga Vasca Cadete,10.000000,5.00000,5.0,5.0,4.0000,8.000000,8.0000,4.000000,8.00000,10.000000,7.600000
4,Aitor Suberviola,Deportivo Alavés Cad. A,251459,93857,Mediocentro,Liga Vasca Cadete,0.758125,0.00000,0.0,10.0,4.0050,1.500000,1.6376,1.500000,5.20205,0.705325,2.640722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215517,Josu Aiizpurua Olabe,Aretxabaleta KE,576664,172798,Mediocentro,Tercera RFEF,6.000000,9.00000,9.0,9.0,8.0000,8.000000,4.0000,4.000000,4.00000,6.000000,7.393103
215518,Roldan Guirao Mikel,Aretxabaleta KE,617138,172798,Mediocentro Defensivo,Tercera RFEF,2.544000,4.91286,10.0,10.0,1.5000,1.500000,10.0000,1.748250,10.00000,1.887000,5.744415
215519,Lasa Sanz Jagoba,Aretxabaleta KE,617139,172798,Mediocentro,Tercera RFEF,0.443810,0.00000,0.0,10.0,1.5000,1.500000,2.0000,1.500000,2.00000,0.332857,1.891753
215520,Sanchez Iparragirre Unax,Aretxabaleta KE,617140,172798,Mediocentro Defensivo,Tercera RFEF,1.082000,4.41119,10.0,10.0,1.7835,0.713400,10.0000,2.215999,10.00000,4.310125,5.852192


In [11]:
# Lista de columnas KPI
kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion',
               'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion',
               'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion',
               'KPI_posicionamiento_tactico', 'KPI_rendimiento']

# Obtener máximos
maximos = df_eventos_con_KPI[kpi_columns].max()

# Función para encontrar el intervalo más común
def find_most_common_interval(series, bins=10):
    hist, bin_edges = np.histogram(series.dropna(), bins=bins)
    max_count_idx = np.argmax(hist)
    return (
        round(bin_edges[max_count_idx], 2),
        round(bin_edges[max_count_idx + 1], 2),
        hist[max_count_idx]
    )

# Calcular intervalos más comunes
intervalos = {}
for col in kpi_columns:
    start, end, count = find_most_common_interval(df_eventos_con_KPI[col])
    intervalos[col] = {
        'intervalo': f'{start} - {end}',
        'frecuencia': count
    }

# Mostrar resultados
print("ANÁLISIS DE KPIs:\n")
print("VALORES MÁXIMOS:")
print("-" * 50)
for col, max_val in maximos.items():
    print(f"{col}: {max_val:.2f}")

print("\nINTERVALOS MÁS FRECUENTES:")
print("-" * 50)
for col, info in intervalos.items():
    print(f"{col}:")
    print(f"  Intervalo más común: {info['intervalo']}")
    print(f"  Frecuencia: {info['frecuencia']}")
    print()

ANÁLISIS DE KPIs:

VALORES MÁXIMOS:
--------------------------------------------------
KPI_condicional: 10.00
KPI_construccion_ataque: 10.00
KPI_progresion: 10.00
KPI_habilidad_individual: 10.00
KPI_peligro_generado: 10.00
KPI_finalizacion: 10.00
KPI_eficacia_defensiva: 10.00
KPI_juego_aereo: 8.00
KPI_capacidad_recuperacion: 10.00
KPI_posicionamiento_tactico: 10.00
KPI_rendimiento: 9.82

INTERVALOS MÁS FRECUENTES:
--------------------------------------------------
KPI_condicional:
  Intervalo más común: 6.0 - 7.0
  Frecuencia: 57717

KPI_construccion_ataque:
  Intervalo más común: 5.0 - 6.0
  Frecuencia: 52945

KPI_progresion:
  Intervalo más común: 5.0 - 6.0
  Frecuencia: 53224

KPI_habilidad_individual:
  Intervalo más común: 5.0 - 6.0
  Frecuencia: 54981

KPI_peligro_generado:
  Intervalo más común: 3.8 - 4.69
  Frecuencia: 56753

KPI_finalizacion:
  Intervalo más común: 5.12 - 6.1
  Frecuencia: 57338

KPI_eficacia_defensiva:
  Intervalo más común: 4.0 - 5.0
  Frecuencia: 54795

KPI

In [12]:
# Definir la ruta del archivo parquet
parquet_path = '../../data_backup/archivos_parquet/df_eventos_acumulados_sin_KPI.parquet'

# Leer el archivo parquet
df_eventos = pd.read_parquet(parquet_path)

# Verificar que se cargó correctamente
print("Dimensiones del DataFrame:", df_eventos.shape)
print("\nColumnas del DataFrame:")
print(df_eventos.columns.tolist())
print("\nPrimeras filas del DataFrame:")
print(df_eventos.head())

Dimensiones del DataFrame: (14650, 180)

Columnas del DataFrame:
['match_id', 'partido', 'jornada', 'team_id', 'equipo', 'player_id', 'season_id', 'temporada', 'liga', 'dorsal', 'demarcacion', 'jugador', 'pases_adelante', 'pases_atras', 'pases_horizontal', 'pases_adelante_inicio', 'pases_adelante_creacion', 'pases_adelante_finalizacion', 'pases_atras_inicio', 'pases_atras_creacion', 'pases_atras_finalizacion', 'pases_horizontal_inicio', 'pases_horizontal_creacion', 'pases_horizontal_finalizacion', 'pases_progresivos_inicio', 'pases_progresivos_creacion', 'pases_progresivos_finalizacion', 'pases_fallidos_campo_propio', 'pases_fallidos_campo_contrario', 'pases_exitosos_campo_propio', 'pases_largos_exitosos', 'pases_largos_fallidos', 'centros_alejados', 'centros_linea_fondo', 'cambios_orientacion_exitosos', 'cambios_orientacion_fallidos', 'pases_recibidos_perfilado', 'POP', 'conducciones_inicio', 'conducciones_creacion', 'conducciones_finalizacion', 'conducciones_carril_izquierdo', 'condu

In [13]:
# Método 1: Ver lista de columnas
print("Lista de columnas:")
print(df_eventos.columns.tolist())

Lista de columnas:
['match_id', 'partido', 'jornada', 'team_id', 'equipo', 'player_id', 'season_id', 'temporada', 'liga', 'dorsal', 'demarcacion', 'jugador', 'pases_adelante', 'pases_atras', 'pases_horizontal', 'pases_adelante_inicio', 'pases_adelante_creacion', 'pases_adelante_finalizacion', 'pases_atras_inicio', 'pases_atras_creacion', 'pases_atras_finalizacion', 'pases_horizontal_inicio', 'pases_horizontal_creacion', 'pases_horizontal_finalizacion', 'pases_progresivos_inicio', 'pases_progresivos_creacion', 'pases_progresivos_finalizacion', 'pases_fallidos_campo_propio', 'pases_fallidos_campo_contrario', 'pases_exitosos_campo_propio', 'pases_largos_exitosos', 'pases_largos_fallidos', 'centros_alejados', 'centros_linea_fondo', 'cambios_orientacion_exitosos', 'cambios_orientacion_fallidos', 'pases_recibidos_perfilado', 'POP', 'conducciones_inicio', 'conducciones_creacion', 'conducciones_finalizacion', 'conducciones_carril_izquierdo', 'conducciones_carril_central', 'conducciones_carril_

In [14]:
df_eventos_con_KPI

,jugador,equipo,player_id,match_id,demarcacion,liga,KPI_condicional,KPI_construccion_ataque,KPI_progresion,KPI_habilidad_individual,KPI_peligro_generado,KPI_finalizacion,KPI_eficacia_defensiva,KPI_juego_aereo,KPI_capacidad_recuperacion,KPI_posicionamiento_tactico,KPI_rendimiento
0,Unax Ariza Fernandez,Deportivo Alavés Cad. A,121382,93857,Mediocentro,Liga Vasca Cadete,6.000000,9.00000,9.0,9.0,8.0000,8.000000,4.0000,4.000000,4.00000,6.000000,7.393103
1,Unax Ariza Fernandez,Deportivo Alavés Cad. A,121382,93857,Mediocentro,Liga Vasca Cadete,6.000000,9.00000,9.0,9.0,8.0000,8.000000,4.0000,4.000000,4.00000,6.000000,7.393103
2,Biniam Gonzalez,Deportivo Alavés Cad. A,251457,93857,Extremo Izquierdo,Liga Vasca Cadete,10.000000,5.00000,5.0,5.0,4.0000,8.000000,8.0000,4.000000,8.00000,10.000000,7.600000
3,Biniam Gonzalez,Deportivo Alavés Cad. A,251457,93857,Extremo Izquierdo,Liga Vasca Cadete,10.000000,5.00000,5.0,5.0,4.0000,8.000000,8.0000,4.000000,8.00000,10.000000,7.600000
4,Aitor Suberviola,Deportivo Alavés Cad. A,251459,93857,Mediocentro,Liga Vasca Cadete,0.758125,0.00000,0.0,10.0,4.0050,1.500000,1.6376,1.500000,5.20205,0.705325,2.640722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215517,Josu Aiizpurua Olabe,Aretxabaleta KE,576664,172798,Mediocentro,Tercera RFEF,6.000000,9.00000,9.0,9.0,8.0000,8.000000,4.0000,4.000000,4.00000,6.000000,7.393103
215518,Roldan Guirao Mikel,Aretxabaleta KE,617138,172798,Mediocentro Defensivo,Tercera RFEF,2.544000,4.91286,10.0,10.0,1.5000,1.500000,10.0000,1.748250,10.00000,1.887000,5.744415
215519,Lasa Sanz Jagoba,Aretxabaleta KE,617139,172798,Mediocentro,Tercera RFEF,0.443810,0.00000,0.0,10.0,1.5000,1.500000,2.0000,1.500000,2.00000,0.332857,1.891753
215520,Sanchez Iparragirre Unax,Aretxabaleta KE,617140,172798,Mediocentro Defensivo,Tercera RFEF,1.082000,4.41119,10.0,10.0,1.7835,0.713400,10.0000,2.215999,10.00000,4.310125,5.852192


In [15]:
# 1. Seleccionar solo las columnas KPI del segundo DataFrame
columnas_kpi = [col for col in df_eventos_con_KPI.columns if 'KPI' in col]
columnas_kpi_mas_keys = ['match_id', 'player_id'] + columnas_kpi

# 2. Crear un DataFrame con solo las columnas que nos interesan y limpiar NaN y 0
df_kpi_filtrado = df_eventos_con_KPI[columnas_kpi_mas_keys]
df_kpi_filtrado = df_kpi_filtrado[
    (df_kpi_filtrado['match_id'].notna()) & 
    (df_kpi_filtrado['player_id'].notna()) &
    (df_kpi_filtrado['match_id'] != 0) & 
    (df_kpi_filtrado['player_id'] != 0)
]

# Limpiar también el DataFrame original
df_eventos_limpio = df_eventos[
    (df_eventos['match_id'].notna()) & 
    (df_eventos['player_id'].notna()) &
    (df_eventos['match_id'] != 0) & 
    (df_eventos['player_id'] != 0)
]

# 3. Hacer el merge
df_final = df_eventos_limpio.merge(
    df_kpi_filtrado,
    on=['match_id', 'player_id'],
    how='left'
)

# 3.5 Eliminar duplicados basados en match_id y player_id
filas_antes = len(df_final)
df_final = df_final.drop_duplicates(subset=['match_id', 'player_id'], keep='first')
filas_despues = len(df_final)

# 4. Guardar el resultado
directory = '../../data_backup/archivos_parquet'
os.makedirs(directory, exist_ok=True)
parquet_path = os.path.join(directory, 'eventos_datos_acumulados.parquet')
df_final.to_parquet(parquet_path, index=False)

# 5. Verificación completa
print(f"Filas originales en df_eventos: {len(df_eventos)}")
print(f"Filas después de eliminar NaN y 0 en df_eventos: {len(df_eventos_limpio)}")
print(f"Filas en df_kpi_filtrado: {len(df_kpi_filtrado)}")
print(f"Columnas KPI añadidas: {len(columnas_kpi)}")
print(f"Filas totales antes de eliminar duplicados: {filas_antes}")
print(f"Filas totales después de eliminar duplicados: {filas_despues}")
print(f"Filas eliminadas por duplicados: {filas_antes - filas_despues}")
print("\nArchivo guardado en:", parquet_path)

Filas originales en df_eventos: 14650
Filas después de eliminar NaN y 0 en df_eventos: 14186
Filas en df_kpi_filtrado: 215461
Columnas KPI añadidas: 11
Filas totales antes de eliminar duplicados: 210830
Filas totales después de eliminar duplicados: 14186
Filas eliminadas por duplicados: 196644

Archivo guardado en: ../../data_backup/archivos_parquet/eventos_datos_acumulados.parquet
